<a href="https://colab.research.google.com/github/worldterminator/mess/blob/main/leap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Rationale

For each year, I keep only the matched records from enrollment and assessment (tho score could be NA) files. Yet across different years, I keep the unmatched student IDs.
Then, each student that remains in the sample should have scores in all their enrolled years (NA included), allowing me to identify cohorts based on first year of enrollment and grade levels.

#Attempt with 16-17 data

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
import pandas as pd

# File path
file_path = '/content/drive/My Drive/Assessment201617.csv'

# Read the CSV file
data= pd.read_csv(file_path)

# Display the first few rows of the DataFrame
data.head()


,TermName,DistrictName,District_StateID,SchoolName,School_StateID,StudentID,Student_StateID,Subject,Course,GrowthMeasureYN,...,ProjectedProficiencyStudy6,ProjectedProficiencyLevel6,ProjectedProficiencyStudy7,ProjectedProficiencyLevel7,ProjectedProficiencyStudy8,ProjectedProficiencyLevel8,ProjectedProficiencyStudy9,ProjectedProficiencyLevel9,ProjectedProficiencyStudy10,ProjectedProficiencyLevel10
0,Spring 2016-2017,Leap Academy University Charter School,807109,STEM Lower Elementary,NaN,1002,1591820143.000,Mathematics,Math K-12,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spring 2016-2017,Leap Academy University Charter School,807109,STEM Lower Elementary,NaN,1002,1591820143.000,Language Arts,Reading,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Spring 2016-2017,Leap Academy University Charter School,807109,STEM Lower Elementary,NaN,1003,7816959131.000,Mathematics,Math K-12,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Spring 2016-2017,Leap Academy University Charter School,807109,STEM Lower Elementary,NaN,1003,7816959131.000,Language Arts,Reading,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Spring 2016-2017,Leap Academy University Charter School,807109,STEM Lower Elementary,NaN,1005,6200882279.000,Mathematics,Math K-12,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


List of column names you want to drop
columns_to_drop = ['column_name4', 'column_name5']

Drop these columns
data.drop(columns=columns_to_drop, inplace=True)

In [69]:
# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'TestName','TestRITScore','LexileScore']

# Create a new DataFrame with only these columns
score1617= data[columns_to_keep]

score1617.head()


,StudentID,Student_StateID,TestName,TestRITScore,LexileScore
0,1002,1591820143.000,Growth: Math K-2 CCSS 2010 V2,213,NaN
1,1002,1591820143.000,Growth: Reading K-2 CCSS 2010,195,511
2,1003,7816959131.000,Growth: Math K-2 CCSS 2010 V2,196,NaN
3,1003,7816959131.000,Growth: Reading K-2 CCSS 2010,183,295
4,1005,6200882279.000,Growth: Math K-2 CCSS 2010 V2,191,NaN


In [70]:
# Set display option to avoid scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [71]:
print(score1617)

# if want to convert state ID: map1617['column_name'] = map1617['column_name'].astype(int)


       StudentID  Student_StateID                          TestName  \
0           1002   1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002   1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003   7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003   7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1005   6200882279.000     Growth: Math K-2 CCSS 2010 V2   
...          ...              ...                               ...   
3066  9982194839              NaN   Growth: Reading 6+ CCSS 2010 V2   
3067  9983529412   9983529412.000     Growth: Math 2-5 CCSS 2010 V2   
3068  9983529412   9983529412.000  Growth: Reading 2-5 CCSS 2010 V2   
3069  9983668608              NaN     Growth: Math 2-5 CCSS 2010 V2   
3070  9983668608              NaN  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  
0              213         NaN  
1              195         511  
2              196         NaN  
3              

Current strategy: join RIT score with enrollment for each year, and then making panels

In [72]:
# File path
file_path = '/content/drive/My Drive/Enrollment201617.csv'

# Read the CSV file
data= pd.read_csv(file_path)

# Display the first few rows of the DataFrame
data.head()

,TermName,DistrictName,District_StateID,SchoolName,School_StateID,StudentID,Student_StateID,StudentDateOfBirth,StudentEthnicGroup,NWEAStandard_EthnicGroup,StudentGender,Grade,NWEAStandard_Grade
0,Spring 2016-2017,Leap Academy University Charter School,807109,STEAM High School,NaN,1018256150,1018256150.000,7/22/2000,Hispanic,Hispanic or Latino,M,11,11
1,Spring 2016-2017,Leap Academy University Charter School,807109,STEAM High School,NaN,1084712619,NaN,12/6/1998,Hispanic,Hispanic or Latino,M,12,12
2,Spring 2016-2017,Leap Academy University Charter School,807109,STEAM High School,NaN,1163227042,NaN,10/31/2000,Hispanic,Hispanic or Latino,F,10,10
3,Spring 2016-2017,Leap Academy University Charter School,807109,STEAM High School,NaN,1183257367,NaN,3/19/1999,Black,Black or African American,M,12,12
4,Spring 2016-2017,Leap Academy University Charter School,807109,STEAM High School,NaN,12272753,6136857128.000,2/29/2000,Hispanic,Hispanic or Latino,F,11,11


In [73]:
# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'StudentEthnicGroup','StudentGender','Grade']

# Create a new DataFrame with only these columns
enrollment1617= data[columns_to_keep]

enrollment1617.head()

,StudentID,Student_StateID,StudentEthnicGroup,StudentGender,Grade
0,1018256150,1018256150.000,Hispanic,M,11
1,1084712619,NaN,Hispanic,M,12
2,1163227042,NaN,Hispanic,F,10
3,1183257367,NaN,Black,M,12
4,12272753,6136857128.000,Hispanic,F,11


In [79]:

# Merge on 'StudentID' with an indicator
merged_df = pd.merge(score1617, enrollment1617, on='StudentID', how='inner', indicator=True)

merged16 = merged_df[merged_df['_merge'] == 'both']

merged16.drop(columns=['_merge'], inplace=True)

tried merging on one of 'student id' and 'state id' sperately and combine the merges and hadnle teh duplicate columns, did not work out well bc multiple observations matched to the same id/combination of ids? not sure, but it produced 100,000 entries

In [80]:
print(merged16)

       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1005     6200882279.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
3066  9982194839                NaN   Growth: Reading 6+ CCSS 2010 V2   
3067  9983529412     9983529412.000     Growth: Math 2-5 CCSS 2010 V2   
3068  9983529412     9983529412.000  Growth: Reading 2-5 CCSS 2010 V2   
3069  9983668608                NaN     Growth: Math 2-5 CCSS 2010 V2   
3070  9983668608                NaN  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              213         NaN     1591820143.000  

In [83]:
# Add a new column named 'Year' with all values set to 2016
merged16['Year'] = 2016

# Display the DataFrame to see the new column
print(merged16)


       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1005     6200882279.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
3066  9982194839                NaN   Growth: Reading 6+ CCSS 2010 V2   
3067  9983529412     9983529412.000     Growth: Math 2-5 CCSS 2010 V2   
3068  9983529412     9983529412.000  Growth: Reading 2-5 CCSS 2010 V2   
3069  9983668608                NaN     Growth: Math 2-5 CCSS 2010 V2   
3070  9983668608                NaN  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              213         NaN     1591820143.000  

#14-15 data

In [89]:
# File path
file_path = '/content/drive/My Drive/assessments201415.csv'

data= pd.read_csv(file_path)
# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'TestName','TestRITScore','LexileScore']

# Create a new DataFrame with only these columns
score1415= data[columns_to_keep]

score1415.head()


,StudentID,Student_StateID,TestName,TestRITScore,LexileScore
0,1002,1591820143.000,Growth: Math K-2 CCSS 2010 V2,170,NaN
1,1002,1591820143.000,Growth: Reading K-2 CCSS 2010,169,BR
2,1003,7816959131.000,Growth: Math K-2 CCSS 2010 V2,154,NaN
3,1003,7816959131.000,Growth: Reading K-2 CCSS 2010,157,BR
4,1008,4297133420.000,Growth: Math K-2 CCSS 2010 V2,157,NaN


In [95]:
# File path
file_path = '/content/drive/My Drive/enrollment201415.csv'

# Read the CSV file
data= pd.read_csv(file_path)

# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'StudentEthnicGroup','StudentGender','Grade']

# Create a new DataFrame with only these columns
enrollment1415= data[columns_to_keep]

enrollment1415.head()

,StudentID,Student_StateID,StudentEthnicGroup,StudentGender,Grade
0,1015267642,1015267642.000,Hispanic,M,6
1,1023348629,NaN,Asian,F,8
2,10483,NaN,Black,F,7
3,10630,NaN,Black,F,6
4,10679,NaN,Hispanic,F,6


In [97]:
# Merge on 'StudentID' with an indicator
merged_df = pd.merge(score1415, enrollment1415, on='StudentID', how='inner', indicator=True)

merged14 = merged_df[merged_df['_merge'] == 'both']

merged14.drop(columns=['_merge'], inplace=True)

print(merged14)

       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1008     4297133420.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
3180  9974474112                NaN      Growth: Math 6+ CCSS 2010 V2   
3181  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
3182  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
3183  9980446488                NaN     Growth: Math 2-5 CCSS 2010 V2   
3184  9980446488                NaN  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              170         NaN     1591820143.000  

In [98]:
# Add a new column named 'Year' with all values set to 2016
merged14['Year'] = 2014

# Display the DataFrame to see the new column
print(merged14)

       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1008     4297133420.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
3180  9974474112                NaN      Growth: Math 6+ CCSS 2010 V2   
3181  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
3182  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
3183  9980446488                NaN     Growth: Math 2-5 CCSS 2010 V2   
3184  9980446488                NaN  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              170         NaN     1591820143.000  

# 15-16 data

In [99]:
# File path
file_path = '/content/drive/My Drive/assessments201516.csv'

data= pd.read_csv(file_path)
# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'TestName','TestRITScore','LexileScore']

# Create a new DataFrame with only these columns
score1516= data[columns_to_keep]

score1516.head()

,StudentID,Student_StateID,TestName,TestRITScore,LexileScore
0,1002,1591820143.000,Growth: Math K-2 CCSS 2010 V2,195,NaN
1,1002,1591820143.000,Growth: Reading K-2 CCSS 2010,187,367
2,1003,7816959131.000,Growth: Math K-2 CCSS 2010 V2,167,NaN
3,1003,7816959131.000,Growth: Reading K-2 CCSS 2010,167,BR
4,1011,8425580900.000,Growth: Math K-2 CCSS 2010 V2,185,NaN


In [100]:
# File path
file_path = '/content/drive/My Drive/enrollment201516.csv'

# Read the CSV file
data= pd.read_csv(file_path)

# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'StudentEthnicGroup','StudentGender','Grade']

# Create a new DataFrame with only these columns
enrollment1516= data[columns_to_keep]

enrollment1516.head()

,StudentID,Student_StateID,StudentEthnicGroup,StudentGender,Grade
0,1018256150,1018256150.000,Hispanic,M,10
1,1046427506,NaN,Hispanic,F,12
2,1084712619,NaN,Hispanic,M,11
3,1088668664,NaN,Hispanic,M,12
4,1121380680,NaN,Hispanic,F,12


In [101]:
# Merge on 'StudentID' with an indicator
merged_df = pd.merge(score1516, enrollment1516, on='StudentID', how='inner', indicator=True)

merged15 = merged_df[merged_df['_merge'] == 'both']

merged15.drop(columns=['_merge'], inplace=True)

print(merged15)

       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1011     8425580900.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
2899  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
2900  9980446488                NaN     Growth: Math 2-5 CCSS 2010 V2   
2901  9980446488                NaN  Growth: Reading 2-5 CCSS 2010 V2   
2902  9983529412     9983529412.000     Growth: Math 2-5 CCSS 2010 V2   
2903  9983529412     9983529412.000  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              195         NaN     1591820143.000  

In [102]:
# Add a new column named 'Year' with all values set to 2016
merged15['Year'] = 2015

# Display the DataFrame to see the new column
print(merged15)

       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1011     8425580900.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
2899  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
2900  9980446488                NaN     Growth: Math 2-5 CCSS 2010 V2   
2901  9980446488                NaN  Growth: Reading 2-5 CCSS 2010 V2   
2902  9983529412     9983529412.000     Growth: Math 2-5 CCSS 2010 V2   
2903  9983529412     9983529412.000  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              195         NaN     1591820143.000  

# 17-18 data

In [103]:
# File path
file_path = '/content/drive/My Drive/assessments201718.csv'

data= pd.read_csv(file_path)
# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'TestName','TestRITScore','LexileScore']

# Create a new DataFrame with only these columns
score1718= data[columns_to_keep]

score1718.head()

,StudentID,Student_StateID,TestName,TestRITScore,LexileScore
0,1001,9784372138,Growth: Math 2-5 NJ 2016,175,NaN
1,1001,9784372138,Growth: Reading 2-5 NJ 2016,168,BR
2,1002,1591820143,Growth: Math 2-5 NJ 2016,211,NaN
3,1002,1591820143,Growth: Reading 2-5 NJ 2016,207,727
4,1003,7816959131,Growth: Math 2-5 NJ 2016,194,NaN


In [112]:
# File path
file_path = '/content/drive/My Drive/enrollment201718.csv'

# Read the CSV file
data= pd.read_csv(file_path)

# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'StudentEthnicGroup','StudentGender','Grade']

# Create a new DataFrame with only these columns
enrollment1718= data[columns_to_keep]

enrollment1718.head()

,StudentID,Student_StateID,StudentEthnicGroup,StudentGender,Grade
0,1015267642,1015267642,Hispanic,M,9
1,1018256150,1018256150,Hispanic,M,12
2,12272794,8032207693,Black,F,11
3,12272797,1325317881,Black,M,11
4,12272798,1021430407,Black,F,11


In [113]:
# Merge on 'StudentID' with an indicator
merged_df = pd.merge(score1718, enrollment1718, on='StudentID', how='inner', indicator=True)

merged17 = merged_df[merged_df['_merge'] == 'both']

merged17.drop(columns=['_merge'], inplace=True)

print(merged17)

       StudentID  Student_StateID_x                     TestName  \
0           1001         9784372138     Growth: Math 2-5 NJ 2016   
1           1001         9784372138  Growth: Reading 2-5 NJ 2016   
2           1002         1591820143     Growth: Math 2-5 NJ 2016   
3           1002         1591820143  Growth: Reading 2-5 NJ 2016   
4           1003         7816959131     Growth: Math 2-5 NJ 2016   
...          ...                ...                          ...   
3006         994         3356386762  Growth: Reading 2-5 NJ 2016   
3007         996         7232964515     Growth: Math 2-5 NJ 2016   
3008         996         7232964515  Growth: Reading 2-5 NJ 2016   
3009  9977980915         9977980915     Growth: Math K-2 NJ 2016   
3010  9977980915         9977980915  Growth: Reading K-2 NJ 2016   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              175         NaN         9784372138              Black   
1              168          BR         

In [114]:
# Add a new column named 'Year' with all values set to 2016
merged15['Year'] = 2017

# Display the DataFrame to see the new column
print(merged17)

       StudentID  Student_StateID_x                     TestName  \
0           1001         9784372138     Growth: Math 2-5 NJ 2016   
1           1001         9784372138  Growth: Reading 2-5 NJ 2016   
2           1002         1591820143     Growth: Math 2-5 NJ 2016   
3           1002         1591820143  Growth: Reading 2-5 NJ 2016   
4           1003         7816959131     Growth: Math 2-5 NJ 2016   
...          ...                ...                          ...   
3006         994         3356386762  Growth: Reading 2-5 NJ 2016   
3007         996         7232964515     Growth: Math 2-5 NJ 2016   
3008         996         7232964515  Growth: Reading 2-5 NJ 2016   
3009  9977980915         9977980915     Growth: Math K-2 NJ 2016   
3010  9977980915         9977980915  Growth: Reading K-2 NJ 2016   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              175         NaN         9784372138              Black   
1              168          BR         